In [1]:
%load_ext autoreload
%autoreload 2


In [2]:

from buttermilk import BM
from promptflow.tracing import trace, start_trace
from rich import print as rprint

import os

from hydra import initialize, compose
from omegaconf import OmegaConf

# Load config, specifying overrides for our particular job
with initialize(version_base=None, config_path="./conf"):
    cfg = compose(config_name='config',
                  overrides=["+data=trans", "+step=trans", "+save=bq", "job=score","source=notebook"])

# Load the main ButterMilk singleton instance
# This takes care of credentials, save paths, and other defaults
bm = BM(cfg=cfg)

# Print config
rprint(OmegaConf.to_container(bm.cfg, resolve=True))



/opt/homebrew/Caskroom/miniconda/base/envs/bm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-11-05 22:20:08 J5HW6L4KT6 buttermilk buttermilk.py[ 297] INFO {'message': "Logging setup for: 20241105T1220Z-8xe7-J5HW6L4KT6-suzor. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'automod', 'job': 'score', 'task_id': '20241105T1220Z-8xe7-J5HW6L4KT6-suzor'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/automod/score/20241105T1220Z-8xe7-J5HW6L4KT6-suzor", 'project': 'automod', 'job': 'score', 'run_id': '20241105T1220Z-8xe7-J5HW6L4KT6-suzor', 'save_dir': 'gs://dmrc-analysis/runs/automod/score/20241105T1220Z-8xe7-J5HW6L4KT6-suzor', 'ip': '159.196.210.27', 'node_name': 'J5HW6L4KT6', 'username': 'suzor'}
2024-11-05 22:20:08 J5HW6L4KT6 buttermilk buttermilk.py[ 305] DEBUG Buttermilk version is: 0.2.0
Prompt flow service has started...


{
    'name': 'automod',
    'job': 'score',
    'source': 'notebook',
    'secret_provider': 'azure',
    'logger': 'gcp',
    'tracing': True,
    'verbose': True,
    'models_secret': 'models',
    'save_dest': 'gcp',
    'save_dir': None,
    'gcp': {'project': 'dmrc-analysis', 'region': 'us-central1', 'bucket': 'dmrc-analysis'},
    'azure': {'vault': 'https://suzorvault.vault.azure.net/', 'resource_group': 'rg-suzor_ai'},
    'aws': {'secret_name': 'secret', 'region': 'us-east-1'},
    'step': [
        {
            'flow': 'judger',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc',
                'template': 'judge',
                'criteria': [
                    'trans_cte',
                    'trans_tja',
                    'trans_glaad',
                    'trans_hrc',
                    'trans_simplified',
                    'trans_factored'
                ],
                'formatting': 'json_rules',
                'model': ['haiku']
            },
            'parameters': None
        },
        {
            'flow': 'synth',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc_judge',
                'template': 'synthesise',
                'criteria': ['trans_simplified'],
                'formatting': 'json_rules',
                'model': ['haiku']
            },
            'parameters': {'answers': 'answers', 'meta': 'meta'},
            'data': {
                'judger': {
                    'name': 'previous runs',
                    'type': 'job',
                    'dataset': 'dmrc-analysis.toxicity.flow',
                    'max_records_per_group': 1,
                    'filter': {'agent_info.flow': 'judger'},
                    'group': {
                        'record_id': 'record.record_id',
                        'flow': 'agent_info.flow',
                        'template': 'agent_info.template',
                        'criteria': 'agent_info.template_vars.criteria',
                        'model': 'parameters.model'
                    },
                    'columns': {
                        'answers': {
                            'id': 'job_id',
                            'model': 'model',
                            'criteria': 'criteria',
                            'template': 'template',
                            'flow': 'flow',
                            'reasons': 'outputs.reasons'
                        },
                        'meta': {
                            'flow': 'agent_info.flow',
                            'template': 'agent_info.template',
                            'criteria': 'agent_info.template_vars.criteria',
                            'model': 'parameters.model',
                            'prediction': 'outputs.prediction',
                            'labels': 'outputs.labels',
                            'confidence': 'outputs.confidence',
                            'timestamp': 'timestamp'
                        }
                    }
                }
            }
        },
        {
            'flow': 'eval',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc',
                'template': 'differences',
                'criteria': ['trans_simplified'],
                'formatting': 'json_reasons',
                'model': ['haiku']
            },
            'parameters': {'answers': 'answers', 'meta': 'meta'},
            'data': {
                'judger': {
                    'name': 'previous runs and synth',
                    'type': 'job',
                    'dataset': 'dmrc-analysis.toxicity.flow',
                    'max_records_per_group': 1,
                    'filter': {'agent_info.flow': ['judger', 'synth']},
                    'group': {
                        'model': 'parameters.model',
                        'record_id': 'rec

## First step assesses content against our criteria

In [ ]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(step=cfg.step[0], data=cfg.data, save=cfg.save, source=cfg.source)


In [ ]:
await orchestrator.run_tasks()


## Second step is a synthesis of previous draft answers

In [ ]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(step=cfg.step[1], data=cfg.data, save=cfg.save, source=cfg.source)


In [ ]:
# Inspect the dataset so we can make sure the inputs look ok
rprint(orchestrator._dataset.sample().answers.values)


In [ ]:
await orchestrator.run_tasks()


## Third step, we summarise the answers, similarities, and differences

In [7]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(step=cfg.step[2], data=cfg.data, save=cfg.save, source=cfg.source)


2024-11-05 22:22:22 J5HW6L4KT6 buttermilk buttermilk.py[ 376] INFO Query stats: Ran in 0:00:03.547215 seconds, cache hit: False, billed 10.49 MB, approx cost $5.2e-06.


INFO:buttermilk:Query stats: Ran in 0:00:03.547215 seconds, cache hit: False, billed 10.49 MB, approx cost $5.2e-06.


<class 'pandas.core.frame.DataFrame'>
Index: 9 entries, NCAA_transgender_athletes to transgender_woman_murder_Baltimore
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   meta     9 non-null      object
 1   answers  9 non-null      object
dtypes: object(2)
memory usage: 216.0+ bytes


In [12]:
# Inspect the dataset so we can make sure the inputs look ok
rprint(orchestrator._dataset.sample().answers.values)


[list([{'id': 'HzHa6MeQuTfJVsfJEipChP', 'flow': 'judger', 'reasons': ["The content provided discusses the 
experiences of a transgender swimmer, Meghan Cortez-Fields, who competed on the women's swim team at Ramapo College
after undergoing hormone therapy and meeting the NCAA's transgender athlete guidelines. The article also examines 
the broader debate around the participation of transgender athletes, particularly transgender women, in women's 
sports at the collegiate and high school levels.", "The key issues raised in the content include the barriers 
transgender people face in accessing healthcare and identity documents, the risks of violence they face, the 
challenges they experience in employment and the criminal justice system, and the ongoing debate over their 
participation in women's sports. The article presents the perspectives of both transgender athletes and their 
advocates, as well as critics who argue that transgender women have unfair physical advantages.", "The content 
acknowledges the lack of consistent, direct research on whether transgender women athletes have a competitive 
advantage over cisgender women, even after undergoing hormone therapy. It cites experts who argue that governing 
bodies like the NCAA should err on the side of inclusivity until more definitive data is available. The article 
also explores the role of Title IX in this debate, with transgender advocates arguing that it should protect the 
rights of transgender athletes, while some cisgender athletes and conservative lawmakers view the inclusion of 
transgender women in women's sports as a violation of Title IX.", 'Overall, the content provides a nuanced and 
balanced examination of the complex issues surrounding transgender athlete participation, highlighting the personal
experiences of transgender athletes, the scientific and legal debates, and the broader social and political 
context.', 'Based on the content and the criteria provided, I conclude that the article does not violate any of the
guidelines for covering transgender people generally. The article uses the correct terminology, respects the 
pronouns and identity of the transgender athlete featured, and provides a comprehensive and contextual analysis of 
the issues surrounding transgender athlete participation.'], 'model': 'haiku', 'criteria': 'trans_cte', 'template':
'judge'}, {'id': 'K8NFNUvLRPksNZbnv69EFh', 'flow': 'judger', 'reasons': ["The content provides context and 
background on the experiences and perspectives of transgender athletes, particularly Meghan Cortez-Fields, a 
competitive swimmer. It explores the debates and policies surrounding transgender participation in women's sports, 
including the NCAA's policy, concerns raised by groups like the Independent Council on Women's Sports, and the lack
of scientific consensus on transgender athletes' potential physical advantages. The content includes quotes and 
reasoning from advocates for transgender inclusion as well as those who oppose it, presenting a balanced 
perspective on this complex and contentious issue.", "The content does not violate the policy criteria on 
'Unnecessary Identification' or 'Uses Incorrect Name or Pronouns'. Meghan Cortez-Fields' transgender identity and 
chosen name and pronouns are presented respectfully and appropriately in the context of discussing her experiences 
as a transgender athlete.", "The content also does not violate the policy criteria on 'Uses Inaccurate 
Phrases/Descriptors'. The language used, such as 'transgender women' and 'cisgender women', is accurate and 
respectful. The article avoids using outdated or offensive terms and correctly distinguishes between gender 
identity, gender expression, and sexual orientation.", "Regarding the 'Uses Politicized Phrases' criteria, the 
content does mention some terms that have been politicized, such as 'gender ideology' and 'trans agenda', but these
are presented as part of the ongoing debate and are not used by the author in a 

In [13]:
await orchestrator.run_tasks()


2024-11-05 22:25:47 J5HW6L4KT6 buttermilk orchestrator.py[  81] DEBUG Starting task for Agent lc_6rrMZx with job 2o8diYaVr52ontPLqrNjbx.


DEBUG:buttermilk:Starting task for Agent lc_6rrMZx with job 2o8diYaVr52ontPLqrNjbx.


2024-11-05 22:26:40 J5HW6L4KT6 buttermilk lc.py[ 201] DEBUG Invoking chain with haiku...


DEBUG:buttermilk:Invoking chain with haiku...


2024-11-05 22:26:41 J5HW6L4KT6 buttermilk orchestrator.py[  81] DEBUG Starting task for Agent lc_6rrMZx with job 7ysAGCxFW2whnNUG6WkerS.


...

DEBUG:buttermilk:Agent lc_6rrMZx completed job ZUrAqJPNFMSEJr5xYawmsd successfully.


2024-11-05 22:28:46 J5HW6L4KT6 buttermilk orchestrator.py[ 106] INFO Completed run 1 of 1


INFO:buttermilk:Completed run 1 of 1


2024-11-05 22:28:46 J5HW6L4KT6 buttermilk orchestrator.py[ 110] INFO All tasks have completed.


INFO:buttermilk:All tasks have completed.
